In [ ]:
# Top squares -> :10 - 1:45
# Top arcos -> 1:47 - 2:47
# bottom squares -> 2:50 - 3:50
# bottom arcos -> 3:53 - 4:46

In [ ]:
import cv2
import numpy as np  
%matplotlib inline
from matplotlib import pyplot as plt
from mirage.mirage_helpers import *

def showim(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()

def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, int(np.ceil(n_images/float(cols))), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

def GetFrameFromVideo(filepath, framenumber):
    _cap = cv2.VideoCapture(filepath)
    _cap.set(cv2.CAP_PROP_POS_FRAMES, framenumber)
    ret, frame = _cap.read()
    return ret, frame

number_of_squares_X = 10
number_of_squares_y = 7
nX = number_of_squares_X - 1 
nY = number_of_squares_y - 1
square_size = 0.025 # 1 inch in meters

# For calibrating front camera
camera_angle = "front"
filepath = "../../top_square.mp4"

# for calibrating side camera
# camera_angle = "side"
# filepath = "../../bottom_squares.mp4"

def crophelpr(img):
    if camera_angle == "front":
        frame = crop_image(img, 0, 720, 0, 1280, 0)
    else:
        frame = crop_image(img, 720, 720, 0, 1280, 0)
    return frame


In [ ]:
### Example Read
# cap = cv2.VideoCapture(filepath)
# flag, frame = cap.read()
# frame = crophelpr(frame)
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# success, corners = cv2.findChessboardCorners(gray, (nY, nX), None)
# chessboard = frame.copy()
# cv2.drawChessboardCorners(chessboard, (nY, nX), corners, success)
# print(success)
# show_images([frame, gray, chessboard], 3)

# Calibration routine

In [ ]:
# store vector of 3D points for all chessboard images ( World coordinant frame )
object_points = []
# store vector of 2D points for all chessboard images ( Camera coordinant frame )
image_points = []

# set termination criteria, we stop when accuracy is reached or a certain number of iterations
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Define real world coordinants for points in the 3D coordinant frame
# Object points are (0, 0, 0,), (1, 0, 0), (2, 0, 0), ....., (5, 8, 0) ?
object_points_3D = np.zeros((nX * nY, 3), np.float32)

# these are the x and y coordinants
object_points_3D[:,:2] = np.mgrid[0:nY, 0:nX].T.reshape(-1,2)

object_points_3D = object_points_3D * square_size


In [ ]:
# for image in images:  # or while loop over the video.
from tqdm.notebook import tqdm
cap = cv2.VideoCapture(filepath)
framelist = []
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
current_frame = 0
usable_checkboards = 0
for framenum in tqdm(range(length)):
    ret, frame = cap.read()
    if not ret:
        cap.release()
        break
    frame = crophelpr(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    success, corners = cv2.findChessboardCorners(gray, (nY, nX), None)
    if success:
        object_points.append(object_points_3D)
        corners_2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        image_points.append(corners_2)
        usable_checkboards += 1

# Calibrate here using above chessboards
print(f"Performing calibration using {usable_checkboards} images") 
height, width = frame.shape[:2]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)
optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))

mean_error = 0
for i in tqdm(range(len(object_points))):
    imgpoints2, _ = cv2.projectPoints(object_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(image_points[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( f"Total usable images {usable_checkboards}")
print( f"total error: {mean_error/len(object_points)}")

# Test using a frame

In [ ]:
# distframenum = 55
# ret, distorted_image = GetFrameFromVideo(filepath, distframenum)
# distorted_image = crophelpr(distorted_image)
# undistorted_image = cv2.undistort(distorted_image, mtx, dist, None, optimal_camera_matrix)

# graydist = cv2.cvtColor(distorted_image, cv2.COLOR_BGR2GRAY)
# success, cornersdist = cv2.findChessboardCorners(graydist, (nY, nX), None)
# chessboarddist = distorted_image.copy()
# cv2.drawChessboardCorners(chessboarddist, (nY, nX), cornersdist, success)


# grayundist = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2GRAY)
# success, cornersundist = cv2.findChessboardCorners(grayundist, (nY, nX), None)
# chessboardundist = undistorted_image.copy()
# cv2.drawChessboardCorners(chessboardundist, (nY, nX), cornersundist, success)

# show_images([distorted_image, undistorted_image, chessboarddist, chessboardundist], 2, ["dist", "undist", "cdist", "cundist"])
# cv2.imwrite("calibOut/a.png", distorted_image)
# cv2.imwrite("calibOut/b.png", undistorted_image)
# cv2.imwrite("calibOut/c.png", chessboarddist)
# cv2.imwrite("calibOut/d.png", chessboardundist)

# Save the calibration

In [ ]:
import pickle
calib_result_pickle = {}
calib_result_pickle["mtx"] = mtx
calib_result_pickle["optimal_camera_matrix"] = optimal_camera_matrix
calib_result_pickle["dist"] = dist
calib_result_pickle["rvecs"] = rvecs
calib_result_pickle["tvecs"] = tvecs
pickle.dump(calib_result_pickle, open(f"{camera_angle}_camera_calib_pickle.p", "wb"))